# Na channels: inactivation

#### **Basic instructions:**
#### 1. **To run code cells**, you can either hit the play button to the left of the cell, or you can select the cell and **press shift-enter**.

#### 2. **The first time** you run this code notebook, you might get a popup asking to choose which version of Python to use (the python "kernel"). **Just hit enter** to choose the base/default version.

#### 3. Make sure you data (.abf) files are in the "data" folder here on the left. You can just copy/paste the files over from where they are saved on your computer.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils import *
update_plot_defaults()
# %load_ext autoreload
# %autoreload 2

## 1. Choose the data file you want to analyze

#### Put the .abf files with your Na channel recordings in the "data/Na_channels" folder

In [ ]:
data_folder = "data/1-Na_channels"

from glob import glob
data_files = glob(data_folder+"/*.abf")
print(data_folder)
data_files

Choose which file you want to analyze and past the file name here:

In [ ]:
data_file = 'data/1-Na_channels/nav18-ssinact.abf'

Now we can load the file and plot the raw data:

In [ ]:
traces = Trace.from_axon_file(filename=data_file, 
                              load_voltage=True, 
                              load_ttl=True,
                              units=['pA', 'mV', 'V'], 
                              concatenate_sweeps=False)
print(traces)

%matplotlib inline
traces.plot(plot_voltage=True, sweep='all', time_units='ms')
plt.show()

## 2. Select what you want to measure

### 2.1. Defining the time window

First let's define the relevant time window where we want to measure the currents:

(If you want to save the images, you can do it by clicking on the save icon in the top-right of the image when you hover over it)

In [ ]:
# The start/end time just helps to zoom in on the traces.
start_time = 490
end_time = 520

# The markers are important: the will define the window where we actually exctract peak current values from. Set them carefully.
marker_1 = 499
marker_2 = 502
time_units = 'ms'

# ----------------------------------------------------------------------------------------------------------------------
%matplotlib widget
ax1,ax2 = traces.plot(plot_voltage=True, time_units=time_units, marker_1=marker_1, marker_2=marker_2, sweep='all')
ax1.set_xlim(start_time,end_time)
ax2.set_xlim(start_time,end_time)
plt.show()

### 3.1. Optional: apply baseline correction

If your baseline current is not zero, you may need to correct for that to get accurate measurements.

Uncomment this cell block (highlight everything, then cmd+/ on macOS or ctrl+/ on Windows) to run it.

In [ ]:
# Change this to True if you want to subtract the baseline from the sweeps.
subtract_baseline = True

In [ ]:
if subtract_baseline:
    traces.subtract_baseline(start_time = 0, 
                             end_time = 0.1, 
                             time_units = time_units,  # specify seconds (s), or milliseconds (ms)
                             channel = 'current')  # Options: 'current', 'voltage', 'all'
    ax1, ax2 = traces.plot(plot_voltage=True, plot_ttl=False, time_units=time_units, sweep='all')
    ax1.set_title('After baseline subtraction', y=0.98)
    plt.show()
else:
    print("BASELINE NOT SUBTRACTED")


### 3.2. Measure currents

Here we will be using the timepoints from the markers we defined above.

First we want to measure the holding voltages from the first phase of the step protocol:


In [ ]:
# Define the timepoint where the voltage is measured
voltage_timepoint = 490 

# ------------------------------------------------------------------------------------------------
_,voltage_steps = traces.get_measurements(start_time=voltage_timepoint, end_time=voltage_timepoint+1, time_units=time_units, sweep='all')

Now we can actually measure the currents and normalized againts the max current.

In [ ]:
measurement_type = 'peak' # Options: 'mean', 'max', 'min', 'peak'

current_steps,_ = traces.get_measurements(start_time=marker_1, end_time=marker_2, 
                                        measurement_type=measurement_type, time_units=time_units)

# Normalize to the biggest current
current_steps /= np.min(current_steps)  # Normalize current steps to the minimum value

# Plot the normalized current steps against the holding voltages
%matplotlib inline
fig,ax = plt.subplots(figsize=(6, 5))
ax.plot(voltage_steps, current_steps, 'o', color='black', markersize=5)
ax.set_xlabel('Holding Voltage (mV)')
ax.set_ylabel('Current amplitude (normalized)')
ax.set_title('Steady-state inactivation curve')

# Optional: fit a Boltzmann function (sigmoid curve) to the data
from scipy.optimize import curve_fit
def boltzmann(V, V_half, k):
    return 1 / (1 + np.exp((V_half - V) / k))
k_guess = -5 # Initial guess for the slope parameter
popt, _ = curve_fit(boltzmann, voltage_steps, current_steps, p0=[np.median(voltage_steps), k_guess])
ax.plot(voltage_steps, boltzmann(voltage_steps, *popt), linestyle='--', color='g', label='Fit')
print("Boltzmann Fit parameters:")
print(f"V_half: {popt[0]:.3f}, k: {popt[1]:.3f}")
plt.show()